# Lesson 2: Prompting o1

In [47]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [48]:
import os
from IPython.display import display, Markdown, HTML
from openai import AzureOpenAI
from dotenv import load_dotenv

load_dotenv()

client = AzureOpenAI(
        azure_endpoint = os.environ.get("AZURE_OPENAI_API_ENDPOINT"),
        api_key = os.getenv("AZURE_OPENAI_API_KEY"),  
        api_version = os.getenv("AZURE_OPENAI_API_VERSION"),
        azure_deployment = os.getenv("AZURE_OPENAI_DEPLOYMENT")
    )
O1_MODEL = os.getenv('AZURE_OPENAI_MODEL')  # Using mini for speed and cost

## Principles

### 1. Simple and direct

### 2. No explicit CoT required

The first principles we start with are **simple and direct** prompting and avoiding providing **explicit guidance or CoT**. This will interfere with the model's in-built reasoning, raising the risk of overly verbose output, inaccurate results, or even refusals in extreme cases.

In [49]:
bad_prompt = ("Generate a function that outputs the SMILES IDs for all the molecules involved in insulin."
              "Think through this step by step, and don't skip any steps:"
              "- Identify all the molecules involve in insulin"
              "- Make the function"
              "- Loop through each molecule, outputting each into the function and returning a SMILES ID"
              "Molecules: ")
response = client.chat.completions.create(model=O1_MODEL,messages=[{"role":"user","content": bad_prompt}])

In [50]:
display(HTML('<div style="background-color: #f0fff8; padding: 10px; border-radius: 5px; border: 1px solid #d3d3d3;"></hr><h2>🔽 &nbsp; Markdown Output – Beginning</h2></hr></div>'))
display(Markdown(response.choices[0].message.content))
display(HTML('<div style="background-color: #fff4f4; padding: 10px; border-radius: 5px; border: 1px solid #d3d3d3;"></hr><h2>🔼 &nbsp; Markdown Output – End</h2></hr></div>'))

Below is an illustrative approach to creating a Python function that returns SMILES strings for the component molecules of insulin. Because insulin is actually a large peptide hormone (rather than a single small molecule), there isn’t one universal SMILES string for “insulin” per se. Instead, one can consider the individual amino acids and important bridging components such as disulfide bonds or other small-molecule groups (e.g., if there are salt counterions, water molecules in crystal forms, etc.). 

Note: This example uses a simplified assumption that “the molecules involved in insulin” refers to the individual standard amino acids that form insulin, as well as highlighting that actual insulin has disulfide bonds linking the A and B chains. If you need SMILES for the complete polypeptide (the entire insulin chain as a single SMILES), you would need specialized cheminformatics tools because it is quite large. 

----------------------------------------------------------------------------
1) Identify the molecules involved in insulin
----------------------------------------------------------------------------
• Human insulin consists of 51 amino acids:
  – A chain: 21 amino acids 
  – B chain: 30 amino acids
• The fundamental “building blocks” are the 20 standard amino acids.  
• There are also three disulfide bonds, but disulfide bonds don’t typically have a separate SMILES representation apart from the cysteine residues themselves in the chain.

----------------------------------------------------------------------------
2) Plan to store or generate SMILES for the molecules
----------------------------------------------------------------------------
• You can store standard SMILES for the 20 canonical amino acids.  
• In practice, each residue within a peptide has a slightly modified structure (bonded at N-terminus and C-terminus to adjacent amino acids), so the “free amino acid” SMILES is a simpler representation.

Below is a minimal dictionary of SMILES for the 20 standard free amino acids (one of many possible standard sets). You can add or revise them as needed:

amino_acid_smiles = {
    "Ala": "CC(C(=O)O)N",       # Alanine
    "Arg": "NC(CCCN=C(N)N)C(=O)O",  # Arginine
    "Asn": "NC(CC(=O)N)C(=O)O", # Asparagine
    "Asp": "NC(CC(=O)O)C(=O)O", # Aspartic acid
    "Cys": "NC(CS)C(=O)O",      # Cysteine
    "Gln": "NC(CCC(=O)N)C(=O)O",# Glutamine
    "Glu": "NC(CCC(=O)O)C(=O)O",# Glutamic acid
    "Gly": "C(C(=O)O)N",        # Glycine
    "His": "NC(CC1=CNC=N1)C(=O)O",     # Histidine
    "Ile": "CC[C@H](C)[C@H](C(=O)O)N", # Isoleucine
    "Leu": "CC(C)C[C@H](C(=O)O)N",     # Leucine
    "Lys": "NC(CCCCN)C(=O)O",   # Lysine
    "Met": "CSCC[C@H](C(=O)O)N",# Methionine
    "Phe": "NC(CCc1ccccc1)C(=O)O",    # Phenylalanine
    "Pro": "OC(=O)C1CCCN1",     # Proline
    "Ser": "NC(CO)C(=O)O",      # Serine
    "Thr": "NC(C(C)O)C(=O)O",   # Threonine
    "Trp": "NC(CCc1c[nH]c2c1cccc2)C(=O)O",  # Tryptophan
    "Tyr": "NC(CCc1ccc(O)cc1)C(=O)O",      # Tyrosine
    "Val": "CC(C)[C@H](C(=O)O)N"  # Valine
}

----------------------------------------------------------------------------
3) Create a function to loop through these molecules and return their SMILES
----------------------------------------------------------------------------
Below is a Python function that illustrates how you might loop through a predefined dictionary of SMILES. You would adapt this to focus specifically on the amino acids (and possibly any additional small molecules) that appear in insulin’s sequence.

def get_insulin_component_smiles():
    """
    Returns a dictionary of SMILES for each key molecule involved in insulin.
    In this example, we are returning SMILES for the 20 standard amino acids.
    """
    amino_acid_smiles = {
        "Ala": "CC(C(=O)O)N",
        "Arg": "NC(CCCN=C(N)N)C(=O)O",
        "Asn": "NC(CC(=O)N)C(=O)O",
        "Asp": "NC(CC(=O)O)C(=O)O",
        "Cys": "NC(CS)C(=O)O",
        "Gln": "NC(CCC(=O)N)C(=O)O",
        "Glu": "NC(CCC(=O)O)C(=O)O",
        "Gly": "C(C(=O)O)N",
        "His": "NC(CC1=CNC=N1)C(=O)O",
        "Ile": "CC[C@H](C)[C@H](C(=O)O)N",
        "Leu": "CC(C)C[C@H](C(=O)O)N",
        "Lys": "NC(CCCCN)C(=O)O",
        "Met": "CSCC[C@H](C(=O)O)N",
        "Phe": "NC(CCc1ccccc1)C(=O)O",
        "Pro": "OC(=O)C1CCCN1",
        "Ser": "NC(CO)C(=O)O",
        "Thr": "NC(C(C)O)C(=O)O",
        "Trp": "NC(CCc1c[nH]c2c1cccc2)C(=O)O",
        "Tyr": "NC(CCc1ccc(O)cc1)C(=O)O",
        "Val": "CC(C)[C@H](C(=O)O)N"
    }

    # Optionally, if you only want the amino acids actually found in the A and B
    # chains of human insulin, you could filter the dictionary for those that appear.
    
    # For demonstration, return them all:
    return amino_acid_smiles

# Example usage:
if __name__ == "__main__":
    smiles_dict = get_insulin_component_smiles()
    for amino_acid, smiles in smiles_dict.items():
        print(f"{amino_acid}: {smiles}")

----------------------------------------------------------------------------
Additional Notes
----------------------------------------------------------------------------
• In reality, to represent insulin as an entire molecule, you would need a peptide SMILES that accounts for the specific sequence and the disulfide bridges. Generating such a SMILES typically requires a cheminformatics toolkit.  
• If your intention is to capture truly “all molecules” in a biological context (e.g., ions, water, potential modifications), you would expand the dictionary or use an external database lookup. 
• The stepwise procedure here shows how to (1) identify the relevant building blocks (the amino acids), (2) store or look up their SMILES, and (3) write a function that loops through them and returns or prints them.

In [51]:
good_prompt = ("Generate a function that outputs the SMILES IDs for all the molecules involved in insulin.")
response = client.chat.completions.create(model=O1_MODEL,messages=[{"role":"user","content": good_prompt}])

In [52]:
display(HTML('<div style="background-color: #f0fff8; padding: 10px; border-radius: 5px; border: 1px solid #d3d3d3;"></hr><h2>🔽 &nbsp; Markdown Output – Beginning</h2></hr></div>'))
display(Markdown(response.choices[0].message.content))
display(HTML('<div style="background-color: #fff4f4; padding: 10px; border-radius: 5px; border: 1px solid #d3d3d3;"></hr><h2>🔼 &nbsp; Markdown Output – End</h2></hr></div>'))

Below is an illustrative Python example showing how one might programmatically retrieve SMILES strings for insulin (or its variants) from the PubChem database. Insulin is a relatively large protein (51 amino acids in two chains), so it is typically recorded in PubChem either as “human insulin,” “bovine insulin,” or other species-specific entries. Each entry in PubChem has a unique Compound ID (CID). The example below uses the CID for human insulin in PubChem (70678594) as a demo.

--------------------------------------------------------------------------------
Explanation
--------------------------------------------------------------------------------
1. PubChem stores chemical (and some protein) entities under a numeric Compound ID (CID), and provides a REST API called PUG REST.
2. To retrieve SMILES, you typically call:
   https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/cid/<CID>/property/IsomericSMILES/JSON
3. The returned JSON contains a field "IsomericSMILES" for that compound.
4. For different forms/species of insulin, you can replace the CID in the function below with the CID from PubChem that corresponds to the insulin entry of interest (e.g., 7059529 is often used for bovine insulin; 70678594 for human insulin, etc.).

--------------------------------------------------------------------------------
Sample Code
--------------------------------------------------------------------------------

import requests

def get_insulin_smiles(cid=70678594):
    """
    Retrieve the isomeric SMILES string from PubChem for the given insulin CID.

    Parameters:
        cid (int): PubChem Compound ID corresponding to a specific insulin entry.
                   Default 70678594 is often listed as 'human insulin'.

    Returns:
        str: The isomeric SMILES string for the specified insulin compound.
    """
    url = f"https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/cid/{cid}/property/IsomericSMILES/JSON"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        # Extract the SMILES string from the JSON response
        smiles = data["PropertyTable"]["Properties"][0]["IsomericSMILES"]
        return smiles
    else:
        raise ValueError(
            f"Could not retrieve data from PubChem for CID {cid}. "
            f"HTTP status code: {response.status_code}."
        )

# Example usage:
if __name__ == "__main__":
    human_insulin_smiles = get_insulin_smiles(cid=70678594)
    print("Human Insulin SMILES:", human_insulin_smiles)

--------------------------------------------------------------------------------
Retrieving SMILES for Different Variants
--------------------------------------------------------------------------------
• You can change the default “cid” argument to another insulin-related PubChem CID to get SMILES for its analogs or variants.  
• For instance, bovine insulin often appears under CID 7059529.  
• If you want to compare multiple insulin entries, simply call get_insulin_smiles(cid=<some_other_cid>) in a loop or individually.

--------------------------------------------------------------------------------
Note on “All Molecules Involved in Insulin”
--------------------------------------------------------------------------------
• Insulin is one polypeptide composed of two chains (A and B) connected by disulfide bonds; PubChem (and most chemical databases) generally treat each insulin variant as a single macromolecular entity rather than multiple smaller molecules. Thus, you will usually get a single (very long) SMILES for the entire insulin molecule (including both chains).  
• If you need smaller building blocks (e.g., the 51 individual amino acid residues), you would have to parse a Protein Data Bank (PDB) file (e.g., 4INS) using libraries like Biopython or RDKit to extract the SMILES for each residue. That is a more involved task because you must reconstruct each amino acid from the atomic coordinates and account for disulfide bonds, terminal groups, etc.

--------------------------------------------------------------------------------
Summary
--------------------------------------------------------------------------------
Use the function get_insulin_smiles above with the PubChem CID corresponding to the insulin variant of interest to retrieve its SMILES representation. For more granular breakdowns (e.g., amino acids, multiple chains separately), parse a PDB file with a structural biology toolkit (BioPython + RDKit) to generate SMILES for each moiety.

## 3. Use structured formats

Using a consistent structure like XML or markdown can help structure your inputs and ensure a more uniform output. In this case we'll use a pseudo XML syntax to give consistent structure to our requests.

In [53]:
structured_prompt = ("<instructions>You are a customer service assistant for AnyCorp, a provider"
          "of fine storage solutions. Your role is to follow your policy to answer the user's question. "
          "Be kind and respectful at all times.</instructions>\n"
          "<policy>**AnyCorp Customer Service Assistant Policy**\n\n"
            "1. **Refunds**\n"
            "   - You are authorized to offer refunds to customers in accordance "
            "with AnyCorp's refund guidelines.\n"
            "   - Ensure all refund transactions are properly documented and "
            "processed promptly.\n\n"
            "2. **Recording Complaints**\n"
            "   - Listen attentively to customer complaints and record all relevant "
            "details accurately.\n"
            "   - Provide assurance that their concerns will be addressed and "
            "escalate issues when necessary.\n\n"
            "3. **Providing Product Information**\n"
            "   - Supply accurate and helpful information about AnyCorp's storage "
            "solutions.\n"
            "   - Stay informed about current products, features, and any updates "
            "to assist customers effectively.\n\n"
            "4. **Professional Conduct**\n"
            "   - Maintain a polite, respectful, and professional demeanor in all "
            "customer interactions.\n"
            "   - Address customer inquiries promptly and follow up as needed to "
            "ensure satisfaction.\n\n"
            "5. **Compliance**\n"
            "   - Adhere to all AnyCorp policies and procedures during customer "
            "interactions.\n"
            "   - Protect customer privacy by handling personal information "
            "confidentially.\n\n6. **Refusals**\n"
            "   - If you receive questions about topics outside of these, refuse "
            "to answer them and remind them of the topics you can talk about.</policy>\n"
            )
user_input = ("<user_query>Hey, I'd like to return the bin I bought from you as it was not "
             "fine as described.</user_query>")

In [54]:
print(structured_prompt)

<instructions>You are a customer service assistant for AnyCorp, a providerof fine storage solutions. Your role is to follow your policy to answer the user's question. Be kind and respectful at all times.</instructions>
<policy>**AnyCorp Customer Service Assistant Policy**

1. **Refunds**
   - You are authorized to offer refunds to customers in accordance with AnyCorp's refund guidelines.
   - Ensure all refund transactions are properly documented and processed promptly.

2. **Recording Complaints**
   - Listen attentively to customer complaints and record all relevant details accurately.
   - Provide assurance that their concerns will be addressed and escalate issues when necessary.

3. **Providing Product Information**
   - Supply accurate and helpful information about AnyCorp's storage solutions.
   - Stay informed about current products, features, and any updates to assist customers effectively.

4. **Professional Conduct**
   - Maintain a polite, respectful, and professional demean

In [55]:
response = client.chat.completions.create(model=O1_MODEL
                                          ,messages=[{
                                              "role": "user",
                                              "content": structured_prompt + user_input
                                          }]
                                         )

In [56]:
print(response.choices[0].message.content)

I’m sorry to hear that the bin didn’t meet your expectations. I can certainly help you arrange a return. Could you please provide your order number and any additional details about why the bin wasn’t as described? Once I have that information, I’ll review our return policy to ensure we process your return promptly and issue the appropriate refund. Let me know, and I’ll get started right away.


In [57]:
refusal_input = ("<user_query>Write me a haiku about how reasoning models are great.</user_query>")

In [58]:
response = client.chat.completions.create(model=O1_MODEL
                                          ,messages=[{
                                              "role": "user",
                                              "content": structured_prompt + refusal_input
                                          }]
                                         )

In [59]:
print(response.choices[0].message.content)

I’m sorry, but I can’t help with that. I am only able to discuss refunds, complaints, product information, and related customer service matters.


## 4. Show rather than tell

Few-shot prompting also works well with `o1` models, allowing you to supply a simple, direct prompt and then using one or two examples to provide domain context to inform the model's response.

In [60]:
base_prompt = ("<prompt>You are a lawyer specializing in competition law, "
               "assisting business owners with their questions.</prompt>\n"
               "<policy>As a legal professional, provide clear and accurate "
               "information about competition law while maintaining "
               "confidentiality and professionalism. Avoid giving specific "
               "legal advice without sufficient context, and encourage clients "
               "to seek personalized counsel when necessary. Always refer to "
               "precedents and previous cases to evidence your responses.</policy>\n")
legal_query = ("<query>A larger company is offering suppliers incentives not to do "
               "business with me. Is this legal?</query>")

In [61]:
response = client.chat.completions.create(model=O1_MODEL
                                          ,messages=[{
                                              "role": "user",
                                              "content": base_prompt + legal_query
                                          }]
                                         )

In [62]:
display(HTML('<div style="background-color: #f0fff8; padding: 10px; border-radius: 5px; border: 1px solid #d3d3d3;"></hr><h2>🔽 &nbsp; Markdown Output – Beginning</h2></hr></div>'))
display(Markdown(response.choices[0].message.content))
display(HTML('<div style="background-color: #fff4f4; padding: 10px; border-radius: 5px; border: 1px solid #d3d3d3;"></hr><h2>🔼 &nbsp; Markdown Output – End</h2></hr></div>'))

Below is some general information about competition law that may be relevant to your situation. Please note that this information is not individualized legal advice; for specific guidance, you should consult a qualified attorney who can assess the details of your case and jurisdiction.

1. Understanding the Possible Issue (Exclusive Dealing or Boycott)
   • When one company uses its market power (or significant influence) to pressure or incentivize suppliers to stop doing business with a competitor, this may raise competition law concerns.  
   • Such behavior can sometimes be classified as “exclusive dealing” or a “concerted refusal to deal (boycott)” under various antitrust or competition statutes.  

2. Relevant Legal Framework and Precedents
   • In the United States:
     – Section 1 of the Sherman Act may be implicated by agreements that unreasonably restrain trade.  
     – Section 2 of the Sherman Act (monopolization) may be implicated if a dominant firm is engaging in conduct to exclude rivals.  
     – Tampa Electric Co. v. Nashville Coal Co. (1961) is a notable U.S. Supreme Court case discussing exclusive dealing.  
   • In the European Union:
     – Article 101 of the Treaty on the Functioning of the European Union (TFEU) addresses agreements that prevent, restrict, or distort competition.  
     – Article 102 TFEU prohibits abusive conduct by a dominant firm, which can include exclusive supply agreements that foreclose competitors if they lack objective justification.  
     – Cases like Intel v. European Commission have dealt with loyalty rebates and incentives that can be seen as anti-competitive under certain conditions.  

3. Key Factors That Influence Legality
   • Market Power/Dominance: If the larger company has significant market power or is dominant in the relevant market, incentives (financial or otherwise) that pressure suppliers to exclude a competitor are more likely to draw scrutiny from competition authorities.  
   • Harm to Competition: Competition law generally focuses on protecting the competitive process rather than the interests of any one competitor. Authorities will consider whether these incentives substantially reduce your access to crucial inputs or raise barriers to entry for you (or other competitors).  
   • Justifications and Efficiency Gains: Sometimes, a larger company might argue its practices are justified by pro-competitive benefits (e.g., economies of scale, improved distribution). Whether these justifications outweigh potential anti-competitive effects depends on facts such as contract duration, the number of suppliers involved, and the broader economic context.

4. Practical Steps and Recommendations
   • Document the conduct: Keep detailed records of any communications or agreements suggesting suppliers are being incentivized to exclude you.  
   • Assess Market Conditions: Gather information about the larger company’s market share, the competitiveness of your industry, and available alternative suppliers.  
   • Seek Legal Counsel: If you believe the conduct is harming your ability to compete, consulting an attorney experienced in competition law is essential. They can help you evaluate the strength of potential claims and discuss possible remedies (including lodging a complaint with relevant competition authorities).  

Conclusion  
Whether the larger company’s actions are legal often depends on the specifics—if they hold significant market power and their behavior substantially limits competition, then offering incentives to suppliers not to deal with you may violate competition law. However, without knowing all the details of your market structure, contract terms, and jurisdiction, it is impossible to give a definitive answer. You may want to consult with a competition law attorney who can provide tailored guidance based on a thorough review of the facts.

In [63]:
example_prompt = ("<prompt>You are a lawyer specializing in competition law, "
               "assisting business owners with their questions.</prompt>\n"
               "<policy>As a legal professional, provide clear and accurate "
               "information about competition law while maintaining "
               "confidentiality and professionalism. Avoid giving specific "
               "legal advice without sufficient context, and encourage clients "
               "to seek personalized counsel when necessary.</policy>\n"
               """<example>
<question>
I'm considering collaborating with a competitor on a joint marketing campaign. Are there any antitrust issues I should be aware of?
</question>
<response>
Collaborating with a competitor on a joint marketing campaign can raise antitrust concerns under U.S. antitrust laws, particularly the Sherman Antitrust Act of 1890 (15 U.S.C. §§ 1–7). Section 1 of the Sherman Act prohibits any contract, combination, or conspiracy that unreasonably restrains trade or commerce among the states.

**Key Considerations:**

1. **Per Se Illegal Agreements:** Certain collaborations are considered automatically illegal ("per se" violations), such as price-fixing, bid-rigging, and market allocation agreements. For example, in *United States v. Topco Associates, Inc.*, 405 U.S. 596 (1972), the Supreme Court held that market division agreements between competitors are per se illegal under the Sherman Act.

2. **Rule of Reason Analysis:** Collaborations that are not per se illegal are evaluated under the "rule of reason," which assesses whether the pro-competitive benefits outweigh the anti-competitive effects. In *Broadcast Music, Inc. v. Columbia Broadcasting System, Inc.*, 441 U.S. 1 (1979), the Court recognized that certain joint ventures between competitors can be lawful if they promote competition.

3. **Information Sharing Risks:** Sharing competitively sensitive information, such as pricing strategies or customer data, can lead to antitrust violations. The Department of Justice and the Federal Trade Commission caution against exchanges that could facilitate collusion (*Antitrust Guidelines for Collaborations Among Competitors*, 2000).

**Recommendations:**

- **Define the Scope:** Clearly delineate the parameters of the collaboration to focus on the marketing campaign without involving competitive aspects like pricing or market division.
- **Implement Safeguards:** Establish protocols to prevent the exchange of sensitive information that is not essential to the marketing effort.
- **Legal Consultation:** Given the complexities of antitrust laws, consult with a legal professional to ensure the collaboration complies with all legal requirements.

**Conclusion:**

While joint marketing campaigns between competitors are not inherently illegal, they must be structured carefully to avoid antitrust pitfalls. Legal guidance is essential to navigate these issues and to design a collaboration that achieves your business objectives without violating antitrust laws.
</response>
</example>""")

In [64]:
response = client.chat.completions.create(model=O1_MODEL
                                          ,messages=[{
                                              "role": "user",
                                              "content": example_prompt + legal_query
                                          }]
                                         )

In [65]:
display(HTML('<div style="background-color: #f0fff8; padding: 10px; border-radius: 5px; border: 1px solid #d3d3d3;"></hr><h2>🔽 &nbsp; Markdown Output – Beginning</h2></hr></div>'))
display(Markdown(response.choices[0].message.content))
display(HTML('<div style="background-color: #fff4f4; padding: 10px; border-radius: 5px; border: 1px solid #d3d3d3;"></hr><h2>🔼 &nbsp; Markdown Output – End</h2></hr></div>'))

Under U.S. antitrust law, agreements or arrangements designed to exclude competitors from critical supplies may raise concerns under Section 1 and/or Section 2 of the Sherman Act (15 U.S.C. § 1 & 2) or Section 3 of the Clayton Act (15 U.S.C. § 14). Whether it is legal for a larger company to offer incentives to suppliers so they won’t do business with you depends on several key factors:

1. Market Power. If the larger company has significant market power or dominance, exclusive arrangements or incentives preventing suppliers from dealing with rivals can be more suspect under the antitrust laws. In particular:
   • Section 2 of the Sherman Act prohibits monopolization or attempts to monopolize, which can include exclusionary agreements that foreclose competition.  
   • Section 3 of the Clayton Act addresses exclusive dealing arrangements that may have the effect of substantially lessening competition in a given market.

2. Degree of Foreclosure. Courts often look at whether the arrangement forecloses (or cuts off) a substantial portion of the supply or distribution channels that competitors need to compete. If the percentage of the market affected is high, the practice may be considered anti-competitive and violate the law.

3. Pro-Competitive Justifications. Even if the incentives limit your access to suppliers, the larger company may argue it has legitimate business reasons—like ensuring supply stability or encouraging investment. Courts and enforcement agencies evaluate whether these justifications outweigh any harm to competition.

4. Impact on Competition, Not Just on a Single Rival. U.S. competition law focuses on protecting the competitive process rather than any single competitor. The key question is whether this conduct harms competition broadly (e.g., raising barriers to entry or reducing consumer choice and innovation).

Recommendations:
• Document Any Adverse Effects. Keep records of supplier communications and any impacts on your ability to compete (e.g., loss of supply, increased costs).
• Evaluate the Market Context. Consider how many suppliers are in your market and how dependent they are on the larger company.
• Seek Tailored Legal Advice. Because exclusivity and incentive arrangements can be legally complex and heavily fact-specific, it is advisable to consult with a qualified legal professional who can assess your particular situation and jurisdiction in greater detail.

Conclusion:
Offering incentives to suppliers not to do business with a competitor can, under certain circumstances, be an illegal exclusionary practice—particularly if the company offering those incentives wields significant market power and succeeds in meaningfully limiting your access to inputs. To discern whether this situation violates antitrust laws requires a careful, fact-based analysis of market conditions, the nature of the incentives, and the competitive impact. You may want to seek professional legal guidance promptly to evaluate the specifics of your case.